In [1]:
using JLD2
using Statistics
using Revise
using Plots
using Laplacians
using Printf

In [2]:
@load "aniso_all.jl.split02merge02.jld2"
dic1 = copy(dic)

Dict{Any, Any} with 84 entries:
  "lamg_build"        => [307.699, 313.176, 320.566, 330.714, 342.547, 343.402,…
  "acM2C2_err"        => [7.27003e-9, 9.73595e-9, 4.72146e-9, 7.28054e-9, 7.128…
  "hypre_solve"       => [45.04, 45.62, 46.78, 46.92, 52.2, 43.7, 36.62]
  "acM2C2_solve"      => [207.587, 210.196, 260.486, 258.833, 255.363, 214.256,…
  "icc2_tot"          => [169.796, 270.038, 429.897, 616.221, 343.401, 136.37, …
  "acM2C2_tot"        => [434.988, 452.815, 516.206, 521.017, 511.237, 436.765,…
  "ac_lst_build"      => [157.899, 181.262, 207.979, 214.741, 208.287, 148.578,…
  "petsc_hypre_build" => [190.56, 222.003, 267.487, 318.002, 227.366, 200.222, …
  "lamg_its"          => [7.0, 8.0, 8.0, 8.0, 8.0, 7.0, 5.0]
  "lamg_solve"        => [Inf, Inf, Inf, Inf, Inf, Inf, Inf]
  "hypre_its"         => [7.0, 7.0, 7.0, 7.0, 8.0, 7.0, 7.0]
  "cmg_solve"         => [79.2035, 104.105, 108.37, 101.635, 102.716, 101.158, …
  "ac_tot"            => [235.069, 262.799, 284.41, 288.22, 295.

In [3]:
function sd(d,x)
    #scale = exp10(ceil(log10(abs(x)))-d)
    #return round(x/scale)*scale
    return round(x, sigdigits=d)
end        

#constant d = 3
ff(s) = (@sprintf "%.3g" sd(3,s))
nf(s) = (@sprintf "%d" sd(3,s))

ffK(s) = (@sprintf "%.3g" sd(3,s)/1000)
nfK(s) =  round(Int64, sd(3,s)/1000)

ffe8(s) = (@sprintf "%.3g" sd(3,s)*1e8)
ffM(s) = (@sprintf "%.3g" sd(3,s)/1e6)
nfM(s) =  round(Int64, sd(3,s)/1e6)
ffe6(s) = (@sprintf "%.3g" sd(3,s)*1e6)

ffe6 (generic function with 1 method)

In [4]:
function labelTvn(tvn, err)
    if err <= 1e-8
        return ffe6(tvn)
    elseif err < 1e-4
        return string(ffe6(tvn), "\\tnote{*}")
    elseif err < 1
        return string(ffe6(tvn), "\\tnote{**}")
    else
        return "Inf"
    end
end

function printSolverCell(d, solver, i)
    if haskey(d, string(solver, "_tot"))
        itr = d[string(solver, "_its")][i]
        ne = d["ne"][i]
        solveVn = d[string(solver, "_solve")][i] / ne
        totalVn = d[string(solver, "_tot")][i] / ne
        err = d[string(solver, "_err")][i]
        return string("&", nf(itr), "&", labelTvn(solveVn, err), "&", labelTvn(totalVn, err), "&",ffe8(err))
    else
        return "&N/A&N/A&N/A&N/A"
    end
end

function nfffswitch(val)
    if val < 1
        return ff(val)
    else
        return nf(val)
    end
end

function printOneLine(d, solvers, i)
    allCells = [printSolverCell(d, solver, i) for solver in solvers]
    return string(nfffswitch(exp10(i-4)),"&", ffM(d["ne"][i]), "&", ffM(d["nv"][i]), join(allCells), "\\\\")
end

function printAllLines(d, solvers)
    for i in 1:length(d["testName"])
        println(printOneLine(d, solvers, i))
    end
end

printAllLines (generic function with 1 method)

In [5]:
solvers1 = ["ac", "ac-s2m2", "cmg2"]
printAllLines(dic1, solvers1)

0.001&203&29.6&26&0.637&1.16&0.661&19&0.729&1.99&0.871&25&0.539&0.968&0.922\\
0.01&200&28.8&26&0.724&1.32&0.904&20&0.864&2.26&0.592&26&0.564&0.979&0.682\\
0.1&200&28.7&27&0.809&1.42&0.618&20&0.956&2.42&0.681&26&0.586&1&0.759\\
1&200&28.7&26&0.811&1.44&0.906&20&0.994&2.5&0.753&28&0.64&1.05&0.591\\
10&200&28.7&27&0.858&1.48&0.635&20&0.991&2.47&0.61&28&0.761&1.19&0.677\\
100&196&28.6&25&0.741&1.31&0.845&19&0.837&2.12&0.842&28&0.633&1.07&0.637\\
1000&189&29.9&23&0.494&0.956&0.733&18&0.502&1.39&0.454&17&0.379&0.845&0.749\\


In [6]:
solvers2 = ["hypre", "petsc_hypre", "icc2"]
printAllLines(dic1, solvers2)

0.001&203&29.6&7&0.222&0.589&0.192&5&0.677&1.62&0.00366&47&0.757&0.836&0.948\\
0.01&200&28.8&7&0.228&0.583&0.2&5&0.732&1.84&0.00456&81&1.26&1.35&0.981\\
0.1&200&28.7&7&0.234&0.581&0.204&5&0.808&2.15&0.00618&132&2.04&2.15&0.989\\
1&200&28.7&7&0.235&0.587&0.205&6&1.01&2.6&0.000371&189&2.93&3.08&0.955\\
10&200&28.7&8&0.261&0.614&0.103&5&0.638&1.77&0.00414&101&1.58&1.72&0.924\\
100&196&28.6&7&0.223&0.574&0.187&4&0.494&1.52&0.116&35&0.562&0.696&0.763\\
1000&189&29.9&7&0.194&0.476&0.0744&4&0.545&1.41&0.00211&12&0.21&0.348&0.651\\
